# 📥 Ingesta de Datos (Bronze Layer)
## Bitcoin Price API

In [1]:
import sys
import os

# Crear carpeta logs si no existe
os.makedirs("./logs", exist_ok=True)
os.makedirs("./db", exist_ok=True)

sys.path.append(os.path.abspath("../src"))

from ingestion import fetch_bitcoin_price , fetch_bitcoin_hourly_prices
from utils import log_and_print ,save_to_db

btc_df = fetch_bitcoin_hourly_prices()
save_to_db(btc_df, "btc_price_bronze")
log_and_print("Bitcoin price saved to Bronze")

/Users/brayanruiz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[2025-04-07 13:22:47] Bitcoin price saved to Bronze


## Noticias CSV

In [2]:
import pandas as pd
news_df = pd.read_csv("../data/news_api.csv")
save_to_db(news_df, "btc_news_bronze")
log_and_print("News data saved to Bronze")

[2025-04-07 13:22:48] News data saved to Bronze


# 🧼 Limpieza y Normalización (Silver Layer)

In [3]:
import sys
import os

sys.path.append(os.path.abspath("../src"))

from processing import clean_news_data, clean_price_data

btc_price_silver = clean_price_data(btc_df)
save_to_db(btc_price_silver, "btc_price_silver")

news_silver = clean_news_data(news_df)
save_to_db(news_silver, "btc_news_silver")
log_and_print("Data cleaned and saved to Silver")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brayanruiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brayanruiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[2025-04-07 13:22:52] Data cleaned and saved to Silver


# 🧠 Feature Engineering (Gold Layer)

In [4]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from processing import add_indicators, extract_keywords  , compute_aggregated_metrics       

btc_price_gold = add_indicators(btc_price_silver)
save_to_db(btc_price_gold, "btc_price_gold")

news_gold = extract_keywords(news_silver)
save_to_db(news_gold, "btc_news_gold")
log_and_print("Feature engineering complete")

gold_metrics = compute_aggregated_metrics(news_gold)
save_to_db(gold_metrics, "btc_news_metrics_gold")
log_and_print("Aggregated metrics saved to Gold layer")

[2025-04-07 13:22:52] Feature engineering complete
[2025-04-07 13:22:52] Aggregated metrics saved to Gold layer


# 💹 Estrategia de Trading (SMA vs EMA)

In [5]:
btc_trading = btc_price_gold.copy()
btc_trading["signal"] = 0
btc_trading.loc[btc_trading["ema_5"] > btc_trading["sma_5"], "signal"] = 1
btc_trading.loc[btc_trading["ema_5"] < btc_trading["sma_5"], "signal"] = -1
btc_trading["trade_signal"] = btc_trading["signal"].diff().fillna(0)
btc_trading["action"] = btc_trading["trade_signal"].map({1: "BUY", -2: "SELL"}).fillna("HOLD")

save_to_db(btc_trading, "btc_trading_strategy")
log_and_print("Trading strategy saved")

[2025-04-07 13:22:52] Trading strategy saved
